In [86]:
import codecs
import numpy as np
import os
import random

In [116]:
with codecs.open('/home/nakul/Documents/Major/Devnagari-Handwritten-Word-Recongition-with-Deep-Learning/data/full.txt', "r", encoding='utf-8') as f:
    lines = f.readlines()

In [117]:
lines #an array is created

['\ufeff\n',
 'HindiSeg/train/10/259/1.jpg हांगकांग\n',
 'HindiSeg/train/10/259/2.jpg लिलेश्वर\n',
 'HindiSeg/train/10/259/3.jpg आयडीन\n',
 'HindiSeg/train/10/259/4.jpg सिखावन\n',
 'HindiSeg/train/10/259/5.jpg विकिपीडिया\n',
 'HindiSeg/train/10/259/6.jpg याचिका\n',
 'HindiSeg/train/10/259/7.jpg निहलानी\n',
 'HindiSeg/train/10/259/8.jpg चिकित्सक\n',
 'HindiSeg/train/10/259/9.jpg उषा\n',
 'HindiSeg/train/10/259/10.jpg गोविंदम\n',
 'HindiSeg/train/10/259/11.jpg हराकर\n',
 'HindiSeg/train/10/259/12.jpg कहूँ\n',
 'HindiSeg/train/10/259/13.jpg आर्किटेक्ट\n',
 'HindiSeg/train/10/259/14.jpg वार्ष्णेय\n',
 'HindiSeg/train/10/259/15.jpg काले\n',
 'HindiSeg/train/10/259/16.jpg एजुकेशन\n',
 'HindiSeg/train/10/259/17.jpg झटका\n',
 'HindiSeg/train/10/259/18.jpg बोला-मैँने\n',
 'HindiSeg/train/10/259/19.jpg चौरसिया\n',
 'HindiSeg/train/10/259/20.jpg एनीमिज्म\n',
 'HindiSeg/train/10/259/21.jpg बनते\n',
 'HindiSeg/train/10/259/22.jpg बताए\n',
 'HindiSeg/train/10/259/23.jpg घीसादास\n',
 'HindiSeg/train/

In [118]:
lines = [x.strip() for x in lines] #

In [119]:
lines[0]

'\ufeff'

In [120]:
for line in lines:
    # ignore comment line
    if not line or line[0]=='#':
        continue

    lineSplit = line.strip().split(' ')
    if lineSplit[0] == '\ufeff':  # since reading first line give '\ufeff'
        continue

In [121]:
linesplit = lines[1].strip().split(' ')
linesplit[1]

'हांगकांग'

In [122]:
text = ' '.join(linesplit[1])

In [123]:
'T'.join('APPLE')

'ATPTPTLTE'

In [124]:
text

'ह ा ं ग क ा ं ग'

In [125]:
text[2] == text[10]

True

In [126]:
arr = np.random.randn(2,3)
arr.shape

(2, 3)

In [127]:
arr = np.stack(arr,axis=0)

In [49]:
arr

array([[-0.43518377, -0.51228886, -1.663126  ],
       [-0.09223251, -0.32015739,  0.20647132]])

In [1]:
str()

''

In [2]:
str(' ')

' '

In [1]:
class Sample:
    "sample from the dataset"
    def __init__(self, gtText, filePath):
        self.gtText = gtText
        self.filePath = filePath
        
class Batch:
    "batch containing images and ground truth texts"
    def __init__(self, gtTexts, imgs):
        self.imgs = np.stack(imgs, axis=0) #search about stack
        self.gtTexts = gtTexts

In [113]:
class DataLoader:

    def __init__(self, filePath, batchSize, imgSize, maxTextLen):
        "loader for dataset at given location, preprocess images and text according to parameters"

        print(filePath)
        assert filePath[-1]=='/'  # should end with '/'
        print(filePath)
        self.dataAugmentation = False
        self.currIdx = 0
        self.batchSize = batchSize
        self.imgSize = imgSize
        self.samples = []
        # use your own path and trick to read the images with the txt file
        with codecs.open(filePath+'new.txt', "r", encoding='utf-8') as f:
            lines = f.readlines()
            print(lines)
        lines = [x.strip() for x in lines] # removing newline
        print("lines",lines)
        chars = set()
        #print(lines[2]) # just check what in lines
        count = 0
        for line in lines:
            # ignore comment line
            
            if not line or line[0]=='#':
                continue

            lineSplit = line.strip().split(' ')

            if lineSplit[0] == '\ufeff':  # since reading first line give '\ufeff'
                continue

            fileName =  '../' +  lineSplit[0]
            print(fileName)

            # GT text are columns starting at 1
            gtText = self.truncateLabel(' '.join(lineSplit[1]), maxTextLen)
            print(type(gtText))
            chars = chars.union(set(list(gtText)))
            print('gtText',gtText)
            print('chars',chars)

            # check if image is not empty
            print(count)
            if not os.path.getsize(fileName):
                continue
                #bad_samples.append(lineSplit[0] + '.jpg')
                
            count = count+1
            # put sample into list
            self.samples.append(Sample(gtText, fileName))
        for samples in self.samples:
            print(samples.gtText,samples.filePath)    

        print('*********************')

        # split into training and validation set: 90% - 10%
        splitIdx = int(0.90 * len(self.samples))
        self.trainSamples = self.samples[:splitIdx]
        self.validationSamples = self.samples[splitIdx:]

        # put words into lists
        self.trainWords = [x.gtText for x in self.trainSamples]
        self.validationWords = [x.gtText for x in self.validationSamples]

        # number of randomly chosen samples per epoch for training 
        self.numTrainSamplesPerEpoch = 25000 

        # start with train set
        self.trainSet()

        # list of all chars in dataset
        self.charList = sorted(list(chars))
        # print("TrainSet: {}, TestSet: {}".format(len(self.trainSamples), len(self.validationSamples)))
        # print("Total chars: ",len(self.charList))  # length: 109
        # charList = ''.join([str(x) for x in self.charList])
        # print(self.charList)
        # uncomment if new ground truth file is added or training from scratch
        # codecs.open('../model/charList.txt','w', encoding='utf-8').write(self.charList)

    def truncateLabel(self, text, maxTextLen):
        # ctc_loss can't compute loss if it cannot find a mapping between text label and input 
        # labels. Repeat letters cost double because of the blank symbol needing to be inserted.
        # If a too-long label is provided, ctc_loss returns an infinite gradient
        cost = 0
        for i in range(len(text)):
            if i != 0 and text[i] == text[i-1]:
                cost += 2
            else:
                cost += 1
            if cost > maxTextLen:
                return text[:i]
        print('text in truncate label',text)
        return text

    def trainSet(self):
        "switch to randomly chosen subset of training set"
        self.dataAugmentation = True
        self.currIdx = 0
        random.shuffle(self.trainSamples)
        self.samples = self.trainSamples[:self.numTrainSamplesPerEpoch]

    def validationSet(self):
        "switch to validation set"
        self.dataAugmentation = False
        self.currIdx = 0
        self.samples = self.validationSamples

    def getIteratorInfo(self):
        "current batch index and overall number of batches"
        return (self.currIdx // self.batchSize + 1, len(self.samples) // self.batchSize) # // floor division

    def hasNext(self):
        "iterator"
        return self.currIdx + self.batchSize <= len(self.samples)

    def getNext(self):
        "iterator"
        batchRange = range(self.currIdx, self.currIdx + self.batchSize)
        gtTexts = [self.samples[i].gtText for i in batchRange]
        imgs = [preprocess(cv2.imread(self.samples[i].filePath, cv2.IMREAD_GRAYSCALE), self.imgSize, self.dataAugmentation) for i in batchRange]
        self.currIdx += self.batchSize
        return Batch(gtTexts, imgs)

In [114]:
loader = DataLoader('../data/', 25, (128,32), 32)

../data/
../data/
['\n', 'HindiSeg/train/10/259/9.jpg उषा\n', 'HindiSeg/train/10/259/5.jpg विकिपीडिया\n', 'HindiSeg/train/10/259/27.jpg भगवत्स्वरूप\n', 'HindiSeg/train/10/259/28.jpg लम्हा\n', 'HindiSeg/train/10/259/33.jpg मेक\n', 'HindiSeg/train/10/204/7.jpg वीएलएसआई\n']
lines ['', 'HindiSeg/train/10/259/9.jpg उषा', 'HindiSeg/train/10/259/5.jpg विकिपीडिया', 'HindiSeg/train/10/259/27.jpg भगवत्स्वरूप', 'HindiSeg/train/10/259/28.jpg लम्हा', 'HindiSeg/train/10/259/33.jpg मेक', 'HindiSeg/train/10/204/7.jpg वीएलएसआई']
../HindiSeg/train/10/259/9.jpg
text in truncate label उ ष ा
<class 'str'>
gtText उ ष ा
chars {'ष', 'ा', ' ', 'उ'}
0
../HindiSeg/train/10/259/5.jpg
text in truncate label व ि क ि प ी ड ि य ा
<class 'str'>
gtText व ि क ि प ी ड ि य ा
chars {'ा', 'ि', 'क', 'ी', 'ड', 'य', 'उ', 'प', ' ', 'ष', 'व'}
1
../HindiSeg/train/10/259/27.jpg
text in truncate label भ ग व त ् स ् व र ू प
<class 'str'>
gtText भ ग व त ् स ् व र ू प
chars {'ा', 'त', 'क', 'ड', 'प', 'ू', ' ', 'ि', 'व', 'भ', 'ी', 'ग', 

In [20]:
fileName= '../HindiSeg/train/10/254/1.jpg'
if not os.path.getsize(fileName):
    continue

SyntaxError: 'continue' not properly in loop (<ipython-input-20-6f1235a858c1>, line 6)

In [3]:
import argparse

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument("--train", help="train the NN", action="store_true")
parser.add_argument("--validate", help="validate the NN", action="store_true")
parser.add_argument("--beamsearch", help="use beam search instead of best path decoding", action="store_true")
parser.add_argument("--wordbeamsearch", help="use word beam search instead of best path decoding", action="store_true")

_StoreTrueAction(option_strings=['--wordbeamsearch'], dest='wordbeamsearch', nargs=0, const=True, default=False, type=None, choices=None, help='use word beam search instead of best path decoding', metavar=None)

In [13]:
loader = DataLoader('../data/', 25, (128,32), 32)
#DataLoader(FilePaths.fnTrain, Model.batchSize, Model.imgSize, Model.maxTextLen)
# save characters of model for inference mode
open(FilePaths.fnCharList, 'w', encoding='UTF-8').write(str().join(loader.charList))

# save words contained in dataset into file
open(FilePaths.fnCorpus, 'w', encoding='UTF-8').write(str(' ').join(loader.trainWords + loader.validationWords))

usage: ipykernel_launcher.py [-h] [--train] [--validate] [--beamsearch]
                             [--wordbeamsearch]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/nakul/.local/share/jupyter/runtime/kernel-eae879bc-c1bd-4c44-93b9-a86a76eeccb3.json


SystemExit: 2

/home/nakul/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
x = [1,2,3,4,5,6,5,3,6,2]
x

[1, 2, 3, 4, 5, 6, 5, 3, 6, 2]

In [6]:
x[:15]

[1, 2, 3, 4, 5, 6, 5, 3, 6, 2]

In [4]:
texts =  [1,2,3,4,5,6,5,3,6,2]

In [7]:
x = enumerate(texts)

In [9]:
print(list(x))

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 5), (7, 3), (8, 6), (9, 2)]
